# Spark Streaming

Apache Spark es un sistema de procesamiento de datos escalable y tolearnte a fallas que soporta tanto operaciones nativas batch y streaming. **Spark Streaming** es una extensión de la API core de Spark que permite a los ingenieros de datos y data scientists procesar data en real time de varias fuentes como (pero no limitada a) Kafka, Flume y Kinesis. La data procesada puede ser enviada a otros file systems, bases de datos, dashboards, etc.

## DStream o Discretized Stream (Stream Discretizado)

Su abstracción clave es llamada Discretized Stream o más conocido como **DStream**, la cual representa el stream de la data dividida en mini batches. Los DStreams estan construidos sobre los RDDs (resilient distributed dataset), que es la abstracción core de Spark. Estos RDDs son una colección de elementos particionados entre nodos del cluster que pueden ser operados en paralelo. Los RDDs son creados tanto desde un file de Hadoop o cualquier colección creada y transformada por el driver. Los usuarios pueden pedir a Spark que persista los RDDs en memoria para que puedan ser reutilizados de manera eficiente entre operaciones paralelas. Finalmente, los RDDs pueden recuperarse automaticamente de fallas. Esta caracteristica de poder recalcularse se le llama **idempotencia**.

<center>

<img src="https://storage.googleapis.com/humai-datasets/imagenes/big_data_pyspark/5_Streaming/streaming-dstream.png" alt="DStream Image" />

</center>

## Streaming Context

Además de la forma que veremos en este colab, existe otra manera de inicializar aplicaciones de Spark Streaming. De la misma manera que otras aplicaciones utilizan el conocido:

```python
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
```

Las aplicaciones de streaming utilizan un objeto llamado `StreamingContext`. Al código anterior se le agrega este objeto y queda de la siguiente manera:

```python
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 1)
```

Como ven el `StreamingContext` recibe dos argumentos. El primero es el `SparkContext` de la aplicación. El segundo es el numero de segundos con el que Spark va a batchear la data entrante. En este caso, la data va a entrar y cada un segundo va a ejecutar el procesamiento de la aplicación.

El beneficio de tener un objeto como `StreamingContext` es que se pueden utilizar `Receivers`, otro objeto que recibe data asincrónica de Spark, o implementar los propios. En mi caso, yo necesitaba consumir imágenes de cámaras de seguridad entonces implemente un `RTSPReceiver` donde `rtsp` es un protocolo de video.

Los `Receivers` que existen son: `socketTextStream` el cual abre un socket y espera data a través de TCP, `fileStream` el cual mira un file system y espera nuevos archivos, entre otros.

## Integración

Gracias a que Spark Streaming esta montado sobre los RDDs, permite que la integración con otros componentes de Spark como Spark SQL o MLlib sea consistente y logica.

El hecho de que Spark tenga solo un motor de ejecución y un modelo de programación para streaming y batch lleva a algunos beneficios únicos sobre otros modelos de programación streaming:

- Rápida recuperación de fallas
- Mejor balanceo de la carga y uso de recursos
- Combinación de data streaming y estática
- Integración nativa con librerías de procesamiento avanzadas (SQL, Machine Learning, procesamiento de grafos)

<center>

<img src="https://storage.googleapis.com/humai-datasets/imagenes/big_data_pyspark/5_Streaming/Apache-Spark-Streaming-ecosystem-diagram.png" alt="Spark Streaming Integration" />

</center>


## Dependencias

Aquí se instalan las dependencias y descargan los archivos necesarios para correr este colab

In [ ]:
!pip install pyspark==3.2.0
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar
!wget https://downloads.apache.org/kafka/3.4.1/kafka_2.12-3.4.1.tgz
!tar -xzf kafka_2.12-3.4.1.tgz

--2023-10-02 21:25:31--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   3.88K  --.-KB/s    in 0s      

2023-10-02 21:25:31 (69.9 MB/s) - ‘iris.csv.1’ saved [3975/3975]

--2023-10-02 21:25:31--  https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/917

# Kafka

Apache Kafka es un sistema de streaming de eventos distribuido de código abierto utilizado por miles de compañías para data pipielines de alta performance, streaming analytics, integración de datos y aplicaciones *mission critical*.

## Arquitectura de Kafka

### Mensajes

Como mencionamos, Kafka envia mensajes a alta velocidad. Para performance óptima, estos mensajes deben ser pequeños: solo algunas columnas. Los mensajes pueden estar serializados de múltiples maneras como AVRO, Proto, String, Bytes, etc. En este caso vamos a pasar un JSON a string y usar el `StringSerializer`. Luego, en la lectura, vamos a desarmar el json en columnas nuevamente. Los mensajes tienen una clave y un valor. La clave va a indicar a qué *broker*, o máquina dentro del cluster, el mensaje caera; y el valor es el valor del mensaje en sí.

### Topics

Los mensajes caen en *topics* o tópicos. Es la manera para que los usuarios (producers y consumers) sepan a donde tienen que escribir mensajes y de donde tienen que leerlos. Los tópicos tienen un nombre y configuración asociada que veremos a continuación.

### Producers

Los *producers* son los usuarios, aplicaciones o entidades que escriben data al topico. Estos se conectan con los *brokers*, definen el tópico al cuál escribir, arman el mensaje con clave y valor, lo serializan y envian.

### Consumers

Del otro lado de los producers, estan los *consumers*, los cuales escuchan mensajes asincrónicamente y ejecutan acciones con esos mensajes. Estas acciones pueden ser transformaciones, nuevos eventos, acciones dentro de un sistema de eventos, etc. De la misma manera que los producers, los consumers deben definir los brokers, topico y métodos de serialización. Cuando un consumer lee un mensaje del cluster, tiene la opción de hacer un *commit*. Esto le indica al cluster que el mensaje fue leido, de manera que si una operación falla y el commit no se hace, puede hacerse nuevamente más tarde por otro consumer.

### Consumer groups

Los *consumer groups* es la manera que tiene kafka de paralelizar los mensajes. Todos los consumers dentro de un consumer group tienen el mismo id. En vez de tener solo un consumer consumiendo todos los mensajes, puedo tener muchos en paralelo, pero ¿Cómo hago para que no lleguen los mismos mensajes a diferentes consumers? Esta abstracción se encarga de esto. Automáticamente, se distribuyen los mensajes de manera equitativa entre los consumers dentro del grupo para paralelizar de la mejor manera posible. La cantidad de consumer groups es configurable, por lo que la escalabilidad (lineal) también lo es.

<center>

<img src="https://sp-ao.shortpixel.ai/client/to_auto,q_lossy,ret_img,w_2231,h_1541/https://www.instaclustr.com/wp-content/uploads/2018/08/Kongo-Blog-6.3-graph_1.6B-1.png" alt="Kafka Scalability Graph" />

</center>

### Brokers

Un cluster de Kafka esta compuesto de uno o más servers llamados *brokers*. En el contexto de Kafka, un broker es un servidor que puede mantener múltiples tópicos y particiones. Se los identifica con un ID único. Una conexión con cualquier nodo broker implica una conexión con el cluster completo. Si hay más de un broker en el cluster estos no *deben* mantener toda la información de un tópico (a pesar de que si puedan dependiendo del factor de replicación definido).

### Particiones

Los tópicos de Kafka estan divididos en un número configurable de partes llamadas *particiones*. Estas permiten que múltiples consumers puedan leer la data de un topico en particular de forma paralela. Las particiones se separan en orden y el número de particiones se especifica cuando se crea el tópico (puede ser cambiado). Cada broker maneja la data para su partición en particular. Como mencionamos, la clave del mensaje determina en qué partición cae el mensaje. Si la clave no esta definida, se decide de manera round-robin. Tener en cuenta que en la mayoría de los casos va a ser más eficiente pensar una lógica para la clave.

### Réplicas

Las *réplicas* son como backups de las particiones de Kafka. Sirven para asegurar que no haya pérdida de datos en caso de falla o apagado planeado. Las particiones de un tópico son guardadas en más de un broker. Estas copias son las réplicas. Tener en cuenta que a mayor cantidad de replicas, mayor disponibilidad pero mayor latencia. Si se pueden perder mensajes, es mejor no tener replicación. Si es importante que no se pierdan mensajes, mejor tener una replicación que sirva para el caso de negocio.

### Misceláneo

Al ser de código abierto, Kafka tiene muchos plugins y extensiones. Una familia de extensiones se llaman *Conectores*. Estos sirven tanto para consumir data de alguna fuente o escribir data a alguna otra fuente. Los conectores de lectura se les llama *Source* o fuente, y los de escritura son *Sink* o "pileta"/"hundir"/"caer". Existen conectores como `MongoDBSink`, `BigQuerySink`, `Pub Sub Sink y Source`, etc.

## Características principales de Kafka

### Alto throughput

Envía mensajes en una red limitada en throughput utilizando un cluster de máquinas con latencias tan bajas como 2ms.

### Escalable

Kafka permite escalar clusters hasta miles de *brokers*, trillones de mensajes por día, petabytes de data, en cientos de miles de *particiones*. Puede escalar elásticamente en storage y procesamiento.

### Storage permanente

Guarda los streams en un cluster distribuido, durable, y tolerante a fallas (noten las similitudes con Spark).

### Alta disponibilidad

Puede incrementar el tamaño de clusters eficientemente entre *availability zones* o conectar clusters entre diferentes regiones geográficas.

In [ ]:
!./kafka_2.12-3.4.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.4.1/config/zookeeper.properties
!./kafka_2.12-3.4.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.4.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 20
!ps -ef | grep kafka
# iniciando el tópico iris con replicación 1 y 1 partición
!./kafka_2.12-3.4.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic iris
!./kafka_2.12-3.4.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic iris

Waiting for 10 secs until kafka and zookeeper services are up and running
root        1017       1  0 19:40 ?        00:00:07 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.4.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.4.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.4.1/bin/../config/log4j.properties -cp /content/kafka_2.12-3.4.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.4.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.4.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.4.1/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.12-3.4.1/bin/../libs/commons-cli-1.4.jar:/conte

### Imports

In [ ]:
import os
import requests

from uuid import uuid4

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import StructType, DoubleType, StringType, IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf, from_json, to_json, struct, md5
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import (
    MinMaxScaler,
    VectorAssembler,
    OneHotEncoder,
    StringIndexer,
    IndexToString
)

### Creando el cluster de Spark con las dependencias instaladas

En este caso, en vez de usar archivos JAR, estamos especificando los paquetes que necesitamos y Spark se encarga de descargarlos por nosotros (si no estuvieran presentes).

Adicionalmente, se crea el cluster de Spark con `local[*]` para que el cluster decida la cantidad de threads que necesita para correr el notebook.

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.kafka:kafka-clients:2.8.1 --master local[*] pyspark-shell'

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("Spark Streaming") \
    .getOrCreate()
sc = spark.sparkContext

### Importando el dataset

En el siguiente bloque se define el schema. En la mayoría de los casos esto no es necesario, pero como las columnas del dataset `iris.csv` tienen puntos en los nombres: `sepal.width` Spark entiende que es un `Struct` o un objeto y trata de descomponerlo. Como no puede, este falla. Lo que hacemos para solucionar esto es cambiarle el nombre agregando *backticks* (el siguiente caracter: `)

En este caso, vamos a usar la función `cache()` al final de la definición del dataset. Esto sirve para mantener el dataset en memoria y que las operaciones sean mucho más rapidas. Hacemos esto ya que luego vamos a ver como se pueden usar un dataset estático y streaming en conjunto.

In [ ]:
iris_schema = StructType().add('sepal.length', DoubleType()) \
  .add('sepal.width', DoubleType()) \
  .add('petal.length', DoubleType()) \
  .add('petal.width', DoubleType()) \
  .add('variety', StringType())

# renaming columns to remove dot for better compatibility
iris_df = spark.read.format('csv') \
  .schema(iris_schema) \
  .option('header', 'true') \
  .load('iris.csv') \
  .select(
      col('`sepal.width`').alias('sepal_width'),
      col('`sepal.length`').alias('sepal_length'),
      col('`petal.width`').alias('petal_width'),
      col('`petal.length`').alias('petal_length'),
      col('variety')
    ).cache()
iris_df.show()
iris_df.printSchema()

+-----------+------------+-----------+------------+-------+
|sepal_width|sepal_length|petal_width|petal_length|variety|
+-----------+------------+-----------+------------+-------+
|        3.5|         5.1|        0.2|         1.4| Setosa|
|        3.0|         4.9|        0.2|         1.4| Setosa|
|        3.2|         4.7|        0.2|         1.3| Setosa|
|        3.1|         4.6|        0.2|         1.5| Setosa|
|        3.6|         5.0|        0.2|         1.4| Setosa|
|        3.9|         5.4|        0.4|         1.7| Setosa|
|        3.4|         4.6|        0.3|         1.4| Setosa|
|        3.4|         5.0|        0.2|         1.5| Setosa|
|        2.9|         4.4|        0.2|         1.4| Setosa|
|        3.1|         4.9|        0.1|         1.5| Setosa|
|        3.7|         5.4|        0.2|         1.5| Setosa|
|        3.4|         4.8|        0.2|         1.6| Setosa|
|        3.0|         4.8|        0.1|         1.4| Setosa|
|        3.0|         4.3|        0.1|  

## Openscoring y copia de modelos

**NOTA IMPORTANTE**: en los comandos `cp` de las celdas que siguen, deben ponerse la dirección de su drive donde apunte a estos archivos. Los archivos estan disponibles en la carpeta del colab. Para conectar colab con drive, abrir los archivos (botón arriba a la izquierda que es una carpeta) y arriba de todo habrá un ícono de drive. Si se le da click se conecta y se agrega una carpeta llamada **drive** en la dirección `/content/drive`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nohup java -jar /content/openscoring-server-executable-2.1.0.jar --port 8081 &
!sleep 10

nohup: appending output to 'nohup.out'


**NOTA IMPORTANTE**: el path debe ser donde cada uno guardo la carpeta_index_to_class y RandomForestIris.pmml
El link donde encuentran la carpeta index_to_class y el archivo PMML es: https://drive.google.com/drive/folders/135dwjynARvhTEAtdS2dm85aiYqLXLKzR?usp=sharing

In [ ]:
!cp -r ./drive/MyDrive/Other/Humai/Clase_5_Spark_Streaming/index_to_class ./index_to_string
!cp -r ./drive/MyDrive/Other/Humai/Clase_5_Spark_Streaming/RandomForestIris.pmml .

cp: cannot stat './drive/MyDrive/Other/Humai/Clase_5_Spark_Streaming/index_to_class': No such file or directory
cp: cannot stat './drive/MyDrive/Other/Humai/Clase_5_Spark_Streaming/RandomForestIris.pmml': No such file or directory


### Data Locallity o Localidad de la Data

Las personas que diseñaron Spark notaron que es más costoeficiente "mover los cómputos" que "mover la data". Es decir, es más barato ejecutar los computos donde esta la data que mover la data a donde esta el computo. Por eso, no solamente Spark es procesamiento distribuido, sino que usa un patrón crucial para su funcionamiento óptimo. Este es, tener en cuenta la **Localidad de los datos**. Esto significa que los procesamientos que se envían al cluster de Spark, deben intentar poder ser performados por las máquinas en donde la data esta y evitar el *shuffling* (que los datos de una maquina termine en otra, que vimos que es costoso).

Es importante tener esto en cuenta al momento de diseñar un sistema utilizando las tecnologías vistas en este colab. Se podría pensar en una arquitectura con los modelos desplegados en la misma máquina donde esta la data, de esta manera las consultas no saldrían de esta y sería extremadamente rápido, a pesar de que fuera HTTP.

Los invito a considerar diferentes opciones y conversarlas en el discord.

In [ ]:
!curl -X PUT --data-binary @RandomForestIris.pmml -H "Content-type: text/xml" http://localhost:8081/openscoring/model/RandomForestIris

{
  "id" : "RandomForestIris",
  "miningFunction" : "classification",
  "summary" : "Ensemble model",
  "properties" : {
    "created.timestamp" : "2023-10-02T21:26:15.942+00:00",
    "accessed.timestamp" : null,
    "file.size" : 319988,
    "file.checksum" : "4171fc0ca8040702d587d0d539d872d0ce089cf156510c8c0a0edc8ec19d5f13",
    "model.version" : null
  },
  "schema" : {
    "inputFields" : [ {
      "id" : "sepal_length",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "sepal_width",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "petal_length",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "petal_width",
      "dataType" : "double",
      "opType" : "continuous"
    } ],
    "targetFields" : [ {
      "id" : "label",
      "dataType" : "double",
      "opType" : "categorical",
      "values" : [ "0.0", "1.0", "2.0" ]
    } ],
    "outputFields" : [ {
      "id" : "prediction",
    

## Se define la UDF para la inferencia en tiempo real

In [ ]:
def make_model_prediction(sepal_width, sepal_length, petal_width, petal_length):
  body = {
    'id': f'record-{uuid4()}',
    'arguments': {'sepal_width': sepal_width, 'sepal_length': sepal_length,
             'petal_width': petal_width, 'petal_length': petal_length}
          }

  headers = {"Content-type": "application/json"}
  response = requests.post(url='http://localhost:8081/openscoring/model/RandomForestIris', json=body, headers=headers)

  return response.json()['results']['prediction']


make_model_prediction_udf = udf(make_model_prediction)

## Se carga el modelo de `IndexToString` para pasar de la predicción numérica a la clase real

In [ ]:
index_to_class = IndexToString.load('./index_to_string')

In [ ]:
!mkdir results
!mkdir checkpoint

mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘checkpoint’: File exists


## Leyendo de Kafka

En este paso se ejecuta la lectura de Kafka, pero no automáticamente ni directamente, sino que se comienza un proceso que va a leer la data cuando llegue, va a desarmar el json, va a ejecutar la predicción, y va a hacer un join con la data original para ver si se predijo bien o no. Esto se ejecuta de manera asincrónica.

In [ ]:
source_schema = StructType().add('sepal_length', DoubleType()) \
  .add('sepal_width', DoubleType()) \
  .add('petal_length', DoubleType()) \
  .add('petal_width', DoubleType())

streaming_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "iris") \
  .load() \
  .select(from_json(col('value').cast('string'), source_schema).alias('value')) \
  .select(col('value.sepal_length').alias('sepal_length'),
          col('value.sepal_width').alias('sepal_width'),
          col('value.petal_length').alias('petal_length'),
          col('value.petal_width').alias('petal_width')) \
  .select('*', make_model_prediction_udf('sepal_width', 'sepal_length',
                                    'petal_width', 'petal_length') \
          .cast(IntegerType()).alias('prediction')) \
  .join(iris_df, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']) \
  .withColumnRenamed('variety', 'original_class')

streaming_df = index_to_class.transform(streaming_df)

streaming_df.writeStream.outputMode('append').format('json').option('path', 'results').option('header', 'true').option("checkpointLocation", "checkpoint").start()

## Escribiendo a Kafka

Ya iniciado el paso anterior, se escribe a Kafka el dataset completo. Esto llegara al proceso anterior y se hara la predicción. Para ver los resutlados, ir a la carpeta ubicada en `/content/results` y buscar los archivos que comienzan en `part-000...`. Ahi estan los resultados en formato json.

En este caso como clave se eligió el hash de los valores de entrada. La realidad es que no hace diferencia ya que solo hay una partición.

In [ ]:
# !sleep 10

iris_df \
  .sample(fraction=0.1) \
  .select(to_json(struct('sepal_width', 'sepal_length', 'petal_width', 'petal_length')).alias('value')) \
  .withColumn('key', md5('value')) \
  .selectExpr("key", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("topic", "iris") \
  .save()

In [ ]:
"hola" \
" mundo"

'hola mundo'